In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Imports

In [2]:
import os
import pandas as pd
import tqdm
import re
import torch
import json
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import tensorflow as tf
from preprocess import *
from vocabulary import *
from model import *
from model_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\swaga\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Pre Processing Dataset

In [4]:
train_df = pd.read_csv('snli/snli_/snli_1.0_train.csv')
val_df   = pd.read_csv('snli/snli_/snli_1.0_dev.csv')
train_df, val_df = preprocess_text(train_df, val_df)
print(len(train_df))
print(len(val_df))

549361
9842


In [5]:
rw = pd.read_csv('test_data/Transformer_generated_summaries.csv', encoding='utf-8')
rw.drop(['Text'], axis = 1, inplace=True)
rw.rename(columns = {'Original_summary':'sentence1'}, inplace = True)
rw.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)
gold = [0] * 1000
rw.insert(loc=0, column='gold_label', value=gold)
rw = rw[['gold_label', 'sentence1', 'sentence2']]
rw = rw.dropna()
rw.head()

,gold_label,sentence1,sentence2
0,0,4 ex-bank officials booked for cheating bank o...,The CBI booked four former officials of Syndi...
1,0,Supreme Court to go paperless in 6 months: CJI,Chief Justice JS Khehar said the Supreme Cour...
2,0,"At least 3 killed, 30 injured in blast in Sylh...",Explosions targetted at people and police off...
3,0,Why has Reliance been barred from trading in f...,Mukesh Ambani-led Reliance Industries barred ...
4,0,Was stopped from entering my own studio at Tim...,Arnab Goswami said he was told he could not d...


In [6]:
train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
rw['sentence1'] = rw['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw['sentence2'] = rw['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [7]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
print(train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)])
print(val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)])

Empty DataFrame
Columns: [gold_label, sentence1, sentence2]
Index: []
Empty DataFrame
Columns: [gold_label, sentence1, sentence2]
Index: []


In [8]:
train_df.head()

,gold_label,sentence1,sentence2
0,neutral,a person on a horse jumps over a broken down a...,a person is training his horse for a competition
1,contradiction,a person on a horse jumps over a broken down a...,a person is at a diner ordering an omelette
2,entailment,a person on a horse jumps over a broken down a...,a person is outdoors on a horse
3,neutral,children smiling and waving at camera,they are smiling at their parents
4,entailment,children smiling and waving at camera,there are children present


In [9]:
val_df.head()

,gold_label,sentence1,sentence2
0,neutral,two women are embracing while holding to go pa...,the sisters are hugging goodbye while holding ...
1,entailment,two women are embracing while holding to go pa...,two woman are holding packages
2,contradiction,two women are embracing while holding to go pa...,the men are fighting outside a deli
3,entailment,two young children in blue jerseys one with th...,two kids in numbered jerseys wash their hands
4,neutral,two young children in blue jerseys one with th...,two kids at a ballgame wash their hands


In [10]:
train_val_df = pd.concat([train_df, val_df])

In [11]:
sentence_pairs, _ = pair_generator(train_val_df)
rw_sentence_pairs, __ = pair_generator(rw)
train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

labels = set(train_sentence_labels)
print(labels)

tag2idx = {word: i for i, word in enumerate(labels)}
print(tag2idx)

train_labels = [tag2idx[t] for t in train_sentence_labels]
val_labels = [tag2idx[t] for t in val_sentence_labels]

{'contradiction', 'neutral', 'entailment'}
{'contradiction': 0, 'neutral': 1, 'entailment': 2}


In [12]:
vocab = Vocabulary()

for data in [rw_sentence_pairs]:
  for sen in data:
    premise    = sen[0]
    hypothesis = sen[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

for data in [sentence_pairs]:
  for sentence_pair in data:
    premise    = sentence_pair[0]
    hypothesis = sentence_pair[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

print("Vocab size:", len(vocab.word2index))

Vocab size: 35119


In [13]:
# vocab = Vocabulary()
print("Vocab size:", len(vocab.word2index))

Vocab size: 35119


In [14]:
index2word = {}
for wrd, idx in vocab.word2index.items():
    # print(wrd, idx)
    index2word[idx] = wrd

## Model

In [15]:
EPOCHS = 20
BATCH_SIZE = 32
EMBEDDING_SIZE = 300
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 64
LEARNING_RATE = 0.006
STACKED_LAYERS = 2
EMBEDDING_PATH = '../../dataset/google_news/GoogleNews-vectors-negative300.bin'
GLOVE_EMBEDDING = '../../embeddings/glove.6B.300d.txt'

initiate_model_vocab(vocab, tag2idx)

In [16]:
train_data = DataSetLoader(get_pair_indices(vocab, train_sentence_pairs), train_labels)
val_data   = DataSetLoader(get_pair_indices(vocab, val_sentence_pairs), val_labels)

In [17]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

17168 308


In [18]:
embeddings_index = load_embeddings(GLOVE_EMBEDDING)

In [19]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

Embedded count: 28742


(35120, 300)

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [21]:
lstm_model = LSTM(VOCAB_SIZE, HIDDEN_SIZE, TARGET_SIZE, STACKED_LAYERS, weights, True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

LSTM(
  (embedding): Embedding(35120, 300)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=256, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [ ]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in lstm_model.state_dict():
    print(param_tensor, "\t", lstm_model.state_dict()[param_tensor].size())

In [ ]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(lstm_model, 'saved_model_final_')

In [23]:
lstm_model = torch.load('saved_models/saved_model_final_')
lstm_model.eval()

LSTM(
  (embedding): Embedding(35120, 300)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=256, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [24]:
print(lstm_model.output)

Linear(in_features=32, out_features=3, bias=True)


In [25]:
sentence_o = [''] * len(rw)
sentence_p = [''] * len(rw)

sentence_o[0] = "many people like to dance to the music of rock and pop"
sentence_p[0] = "people like rock and roll"
sentence_o[1] = "a girl is dancing on the floor"
sentence_p[1] = "dancing girl"

sentence_o[0] = clean_text(sentence_o[0])
sentence_p[0] = clean_text(sentence_p[0])
sentence_o[1] = clean_text(sentence_o[1])
sentence_p[1] = clean_text(sentence_p[1])

sen_p = []
for i in range(2):
  sen1 = sentence_o[i]
  sen2 = sentence_p[i]
  sen_p.append((sen1, sen2))

sen_p

[('many people like to dance to the music of rock and pop',
  'people like rock and roll'),
 ('a girl is dancing on the floor', 'dancing girl')]

In [26]:
rw_sentence_pairs[0]

('ex bank officials booked for cheating bank of crore',
 'the cbi booked four former officials of syndicate bank and six others for cheating forgery')

In [43]:
id_pairs = get_pair_indices(vocab, rw_sentence_pairs)
# id_pairs = get_pair_indices(vocab, sen_p)
id_pairs[0]

([1, 2, 3, 4, 5, 6, 2, 7, 8],
 [9, 10, 4, 11, 12, 3, 7, 13, 2, 14, 15, 16, 5, 6, 17])

In [44]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

1000


In [45]:
prediction

tensor([[-0.1308,  0.9463, -0.3197],
        [-0.4200,  1.0235, -0.1301],
        [-0.5133,  1.2159, -0.1641],
        ...,
        [ 0.3282,  0.4428, -0.4151],
        [ 0.2177,  0.6922, -0.4887],
        [-0.0440,  0.7216, -0.2341]], device='cuda:0', grad_fn=<AddmmBackward>)

In [46]:
prediction[0]

tensor([-0.1308,  0.9463, -0.3197], device='cuda:0', grad_fn=<SelectBackward>)

In [47]:
rw_sentence_pairs[12]

('thousands march in london to protest against brexit',
 'thousands of people took to the streets in london to protest against the decision to leave the')

In [48]:
soft = torch.log_softmax(prediction, dim=1).argmax(dim=1)
soft.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 1,


In [49]:
ctr = 0
for i in range(len(soft)):
    if soft[i] == 0:
        print(rw_sentence_pairs[i])
        ctr = ctr + 1
print(ctr)

('us brewer millercoors sues hcl technologies for crore', 'millercoors sued hcl tech for million over')
('kangana gifts herself bedroom bungalow on birthday report', 'kangana ranaut gifted herself a three bedroom bungalow on the occasion')
('bsf gets first woman field officer after years', 'tanushree pareek is the first woman field officer in the year')
('will give lakh grant to kailash mansarovar pilgrims yogi', 'yogi adityanath has announced a lakh grant to those')
('want to reshoot the song gerua with ice cream tweets srk', 'shah rukh khan wants to reshoot the song gerua')
('censor board demands removal of phrase mann ki baat in film', 'censor board has demanded removal of phrase mann ki baat')
('sp will choose its next party chief before sept akhilesh', 'akhilesh yadav says samajwadi party will choose its next national')
('russia has no plans to meddle in french election putin', 'russian president vladimir putin said that russia has no intention of meddling in french elections puti